<a href="https://colab.research.google.com/github/Skalwalker/AntiMoneyLaundering/blob/main/anti_money_laundering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
!pip install kedro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%load_ext kedro.ipython

[06/22/23 11:22:00] WARNING  Kedro extension was registered but couldn't find a Kedro project. Make  ]8;id=239583;file:///usr/local/lib/python3.10/dist-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=666615;file:///usr/local/lib/python3.10/dist-packages/kedro/ipython/__init__.py#37\37]8;;\
                             sure you run '%reload_kedro <project_root>'.                                          

In [3]:
!kedro new --starter=pandas-iris


Project Name
Please enter a human readable name for your new project.
Spaces, hyphens, and underscores are allowed.
 [Iris]: Anti Money Laundering

The project name 'Anti Money Laundering' has been applied to: 
- The project title in /content/anti-money-laundering/README.md 
- The folder created for your project in /content/anti-money-laundering 
- The project's python package in /content/anti-money-laundering/src/anti_money_laundering

A best-practice setup includes initialising git and creating a virtual environment before running 'pip install -r src/requirements.txt' to install project-specific dependencies. Refer to the Kedro documentation: https://kedro.readthedocs.io/

Change directory to the project generated in /content/anti-money-laundering by entering 'cd /content/anti-money-laundering'
